In [65]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from html_table_parser import parser_functions


def std_crawler(url):
    driver = webdriver.Chrome(executable_path=r"C:\Users\gihok\chatbot\chromedriver.exe")
    driver.get(url)
    time.sleep(3)
    
    df = pd.DataFrame(columns=['표준번호','표준명(한글)','대체 표준','표준명(영문)','KS원문보기','표준분야','표준구분','제정일','최종개정확인일',
                               '기술심의회','전문위원회','적용범위','표준개발협력기관','담당부처','담당부서','담당자명',
                               '담당자 연락처','국제표준 부합화','인용표준','기술기준','ICS Code','표준이력사항','인증심사기준'])
    while True:
        try:
            # 페이지 넘기기
            # for i in range(3, 14):
            for i in range(3,6):
                driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[3]/div/div/ul/li[{}]/a'.format(i)).click()
                
                time.sleep(3)
                
                # 표준 들어가기
                for j in range(1, 11):
                    driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[2]/table/tbody/tr[{}]/td[2]/a'.format(j)).click()
                    
                    time.sleep(3)
                    
                    #crawling 작업
                    try:
                        dict = {}
                        for z in range(1, 22):
                            # 인용표준 및 기술기준
                            if z == 18:
                                title_1 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[1]'.format(z)).text
                                content_1 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[18]/td[1]').text
                                title_2 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[2]'.format(z)).text
                                content_2 = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[18]/td[2]').text
                                dict[title_1] = [content_1]
                                dict[title_2] = [content_2]
                            
                            #테이블 데이터 처리
                            elif z == 17 or z == 20 or z == 21:
                                title = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[1]'.format(z)).text
                                result_html = driver.page_source
                                soup = BeautifulSoup(result_html, 'html.parser')
                                # 국제표준 부합화
                                if z == 17:
                                    data = soup.find("div", {'class':"table list gray"})
                                #표준이력사항
                                elif z == 20:
                                    data = soup.find_all("div", {'class':"table list gray"})[1]
                                #인증심사기준
                                else:
                                    data = soup.find_all("div", {'class':"table list gray"})[2]
                                    
                                table = parser_functions.make2d(data)
                                df_std = pd.DataFrame(data = table[1:], columns = table[0])
                                dict[title] = [df_std]
                                
                            else:    
                                title = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/th[1]'.format(z)).text
                                content = driver.find_element(By.XPATH, '//*[@id="contents"]/div/div[2]/div[1]/table/tbody/tr[{}]/td[1]'.format(z)).text
                                dict[title] = [content]
                                
                        dict_df = pd.DataFrame(dict)
                        df = pd.concat([df, dict_df], ignore_index=True)
                    
                    except:
                        break
                    
                    driver.back()
                    driver.refresh()
                    time.sleep(3)
                    
            return df    
        except:
            break
        
    return df

In [66]:
df = std_crawler("https://standard.go.kr/KSCI/standardIntro/getStandardSearchList.do?menuId=919&topMenuId=502")

C:\Users\gihok\AppData\Local\Temp\ipykernel_117972\1397614501.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\gihok\chatbot\chromedriver.exe")


In [67]:
df

,표준번호,표준명(한글),대체 표준,표준명(영문),KS원문보기,표준분야,표준구분,제정일,최종개정확인일,기술심의회,...,담당부처,담당부서,담당자명,담당자 연락처,국제표준 부합화,인용표준,기술기준,ICS Code,표준이력사항,인증심사기준
0,KS A 0001,표준서의 서식 및 작성방법,NaN,Rules for the drafting and presentation of Kor...,표준 원문보기,(A) 기본 - (A10) 기본일반,방법,1962-06-26,2021-01-11,기계기본 기술심의회(B),...,국가기술표준원,산업표준혁신과,고정하,--,대응국제표준 부합화수준 0 해당없음 해당없음,147 건,1 건,01.120 (표준화. 일반 규칙),변경일자 구분 고시번호 \ 0 1962-06-2...,제/개정일자 구분 첨부파일 0 ...
1,KS A 0002 (폐지),법정계량 용어 (폐지),,International vocabulary of legal metrology,NaN,(A) 기본 - (A17) 기타,전달,2004-12-24,2019-12-31,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,양승배,043-870-5373,대응국제표준 부합화수준 0 해당없음 해당없음,0 건,2 건,해당없음,변경일자 구분 고시번호 \ 0 2004-12-24 ...,제/개정일자 구분 첨부파일 0 ...
2,KS A 0003 (폐지),법정계량 용어 (폐지),KSA0002,Vocabulary of terms in legal metrology,NaN,(A) 기본 - (A10) 기본일반,전달,2004-12-24,2013-11-05,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,양승배,043-870-5373,대응국제표준 부합화수준 0 관련 자료가 없습니다. ...,0 건,0 건,01.040.17 (계랑 및 측정. 물리현상 (용어)),변경일자 구분 고시번호 \ 0 2004-12-24 ...,제/개정일자 구분 첨부파일 0 ...
3,KS A 0004 (폐지),분석 . 시험의 허용차 통칙 (폐지),,GENERAL RULES FOR PERMISSIBLE TOLERANCE OF CHE...,NaN,(A) 기본 -,전달,1981-12-11,2002-12-30,,...,국가기술표준원,기계융합산업표준과,이경희,043-870-5379,대응국제표준 부합화수준 0 ISO 5725-1 NEQ ...,0 건,8 건,,"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표...",제/개정일자 구분 첨부파일 0 ...
4,KS A 0005,제도 ― 통칙(도면 작성의 일반코드),NaN,TECHNICAL DRAWING-GENERAL CODE OF DRAWING PRAC...,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1969-10-25,2019-12-31,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,김민우,043-870-5376,대응국제표준 부합화수준 0 해당없음 해당없음,87 건,1 건,01.100.00 (기술도안 일반),변경일자 구분 고시번호 \ 0 1969-10-2...,제/개정일자 구분 첨부파일 0 ...
5,KS A 0006,시험 장소의 표준 상태,NaN,STANDARD ATMOSPHERIC CONDITIONS FOR TESTING,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1971-05-22,2019-12-24,기계기본 기술심의회(B),...,국가기술표준원,기계융합산업표준과,김민우,043-870-5376,대응국제표준 부합화수준 0 ISO 554:1976 IDT 일치,0 건,64 건,19.020 (시험 조건 및 절차 일반),변경일자 구분 고시번호 ...,제/개정일자 구분 첨부파일 0 ...
6,KS A 0007 (폐지),음향 용어 ( 기기 ) (폐지),,GLOSSARY OF ACOUSTICAL TERMS (TRANSDUCERS AND ...,NaN,(A) 기본 -,전달,1982-11-04,2001-11-19,전기응용 기술심의회(C),...,국가기술표준원,전기전자정보표준과,김성희,043-870-5365,대응국제표준 부합화수준 0 관련 자료가 없습니다. ...,0 건,0 건,01.040.17 (계랑 및 측정. 물리현상 (용어)) 17.140.00 (음향 및...,"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표...",제/개정일자 구분 첨부파일 0 ...
7,KS A 0009 (폐지),음향 용어 ( 음성 . 청각 . 음악 ) (폐지),,GLOSSARY OF ACOUSTICAL TERMS (HEARING.SPEECH.M...,NaN,(A) 기본 -,전달,1982-11-04,2001-11-19,전기응용 기술심의회(C),...,국가기술표준원,전기전자정보표준과,김성희,043-870-5365,대응국제표준 부합화수준 0 관련 자료가 없습니다. ...,0 건,0 건,01.040.17 (계랑 및 측정. 물리현상 (용어)) 17.140.01 (음향 측...,"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표...",제/개정일자 구분 첨부파일 0 ...
8,KS A 0010,오염 물질 조정 용어,NaN,Glossary of terms for contamination control,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1985-05-08,2020-12-30,산업환경 기술심의회(I),...,국가기술표준원,바이오화학서비스표준과,지민호,043-870-5394,대응국제표준 부합화수준 0 해당없음 해당없음,2 건,0 건,01.040.13 (환경 및 보건. 안전도 (용어)),"변경일자 구분 고시번호 제정,개정,폐지 사유 신구대비표...",제/개정일자 구분 첨부파일 0 ...
9,KS A 0011,물체색의 색 이름,NaN,Names of non - luminous object colours,표준 원문보기,(A) 기본 - (A10) 기본일반,전달,1967-12-30,2020-12-22,문화 기술심의회(A),...,국가기술표준원,바이오화학서비스표준과,박정은,043-870-5401,대응국제표준 부합화수준 0 해당없음 해당없음,6 건,8 건,17.180.20 (빛의 색 및 측정),"변경일자 구분 고시번호 제정,개정,...",제/개정일자 구분 첨부파일 0 ...


In [68]:
df.loc[3]

표준번호                                           KS A 0004 (폐지)
표준명(한글)                                  분석 . 시험의 허용차 통칙 (폐지)
대체 표준                                                        
표준명(영문)     GENERAL RULES FOR PERMISSIBLE TOLERANCE OF CHE...
KS원문보기                                                    NaN
표준분야                                                 (A) 기본 -
표준구분                                                       전달
제정일                                                1981-12-11
최종개정확인일                                            2002-12-30
기술심의회                                                        
전문위원회                                                        
적용범위                                                         
표준개발협력기관                                                     
담당부처                                                  국가기술표준원
담당부서                                                기계융합산업표준과
담당자명                                                      이경희
담당자 연락처 

In [70]:
df.loc[3]['국제표준 부합화']

,대응국제표준,부합화수준
0,ISO 5725-1,NEQ 동일하지 않음
1,ISO 5725-2,NEQ 동일하지 않음
2,ISO 5725-3,NEQ 동일하지 않음
3,ISO 5725-4,NEQ 동일하지 않음
4,ISO 5725-6,NEQ 동일하지 않음


In [71]:
df.loc[3]['표준이력사항']

,변경일자,구분,고시번호,"제정,개정,폐지 사유",신구대비표
0,1981-12-11,개정,811985,,
1,1986-10-27,확인,861819,,
2,1991-03-06,확인,910097,,
3,1996-12-27,확인,960194,,
4,1997-06-19,개정,97122,,
5,2002-12-30,폐지,2002-1794,,


In [72]:
df.loc[3]['인증심사기준']

,제/개정일자,구분,첨부파일
0,관련 자료가 없습니다.,관련 자료가 없습니다.,관련 자료가 없습니다.


In [10]:
import json
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import collections
collections.Callable = collections.abc.Callable
import pymysql
import pandas as pd
import datetime


# json file로 변환하는 함수
def transform_to_json(dic, file_name):
    with open(file_name, 'w', encoding = 'UTF8') as f:
        json.dump(dic, f, ensure_ascii=False, indent = '\t')
        
        
# text 데이터 정제 함수        
def clean_data(string):
    regexed_str = re.sub("[\n]+", " ", string)
    regexed_str = re.sub("[\t]+", " ", string)
    return_string = regexed_str.replace("\n", " ")
    return return_string.strip()


# 테이블 형태 데이터 처리 함수
def small_table_parser(contents_node, header_node):
    total_lst = []
    for contents_idx in range(0, len(contents_node), len(header_node.find_all("th"))):
        sub_dict = {}
        for th in range(len(header_node.find_all("th"))):
            sub_dict[header_node.find_all("th")[th].text] = clean_data(contents_node[contents_idx + th].text)
        total_lst.append(sub_dict)
    return total_lst


# 표준 데이터 크롤러
def std_crawler(whole_dict, lst_page_soup, driver, std_idx):
    page_list_data = lst_page_soup.find_all("tbody")[1]
    row_lst = page_list_data.find_all("tr")
    
    driver.implicitly_wait(100)
    
    for standard in range(std_idx, len(row_lst) + 1):
        
        # data 개수 체크
        num = int(lst_page_soup.select("#tabs-container > div.table.list > table > tbody > tr:nth-of-type({}) > td:nth-of-type(1)".format(standard))[0].text)
        
        driver.implicitly_wait(100)
        
        #crawling 작업
        try:
            driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[2]/table/tbody/tr[{}]/td[2]/a'.format(standard)).click()
            
            std = {}
            
            result_html = driver.page_source
            soup = BeautifulSoup(result_html, 'html.parser')
            
            # 테이블 row 개수
            table_data = soup.find("div", {'class':'table view'})
            table_data = table_data.find('tbody').children

            table_lst = []
            for child in table_data:
                table_lst.append(child)
            for i in table_lst:
                if i == "\n":
                    table_lst.remove(i)
            
                                    
            for row in range(1, len(table_lst) + 1):
                selector = "#contents > div > div.content_inner > div.table.view > table > tbody > "
                title = soup.select(selector + "tr:nth-of-type({}) > th".format(row))[0].text
                
                # 인용표준, 기술기준 데이터 / index text를 비교하는 if 문으로 바꾸기 / 가독성 고려
                if title == "인용표준" or title == "기술기준":
                    title = soup.select(selector + "tr:nth-of-type({}) > th".format(row))[0].text
                    content = soup.select(selector + "tr:nth-of-type({}) > td".format(row))[0].text
                    
                    title_2 = soup.select(selector + "tr:nth-of-type({}) > th:nth-of-type(2)".format(row))[0].text
                    content_2 = soup.select(selector + "tr:nth-of-type({}) > td:nth-of-type(2)".format(row))[0].text
                    
                    
                    content = clean_data(content)
                    content_2 = clean_data(content_2)
                    
                    std[title] = content
                    std[title_2] = content_2
                    
                #테이블 데이터 처리 / 유형별로 파악해서 if 문 바꾸기 / 아예 클래스를 만들어서 기존 항목에 추가 제거의 경우에 처리할 경우 / 테이블 파싱하는 함수 만들기 (colspan 예외 처리도)
                elif title == "국제표준 부합화" or title == "표준이력사항" or title == "인증심사기준":
                    std[title] = []
                    
                    # 국제표준 부합화
                    if title == "국제표준 부합화":
                        sub_dict = {}
                        data = soup.find("div", {'class':"table list gray"})
                        contents = data.find_all("td")
                        
                        if "colspan" in contents[0].attrs:
                            for j in range(int(contents[0].attrs["colspan"])):
                                sub_dict[data.find_all("th")[j].text] = contents[0].text
                            std[title].append(sub_dict)
                            contents = contents[1:]
                            
                        table_data_lst = small_table_parser(contents, data)
                        for tb_data in table_data_lst:
                            std[title].append(tb_data)
                        
                    #표준이력사항
                    elif title == "표준이력사항":
                        data = soup.find_all("div", {'class':"table list gray"})[1]
                        contents = data.find_all("td")
                        
                        table_data_lst = small_table_parser(contents, data)
                        std[title] = table_data_lst
                            
                    #인증심사기준
                    else:
                        sub_dict = {}
                        data = soup.find_all("div", {'class':"table list gray"})[2]
                        contents = data.find_all("td")
                        if len(contents) == 1:
                            for i in range(int(contents[0].attrs["colspan"])):
                                sub_dict[data.find_all("th")[i].text] = contents[0].text
                            std[title].append(sub_dict)
                        
                        else:
                            table_data_lst = small_table_parser(contents, data)
                            std[title] = table_data_lst

                # 나머지 데이터
                else:
                    content = soup.select(selector + "tr:nth-of-type({}) > td".format(row))[0].text
                    content = clean_data(content)
                    std[title] = content
                    
            now = time.localtime()
            now = "%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
            std['crawled_time'] = now
            whole_dict['stds'].append(std)
            
            driver.back()
            driver.refresh()
            
        except:
            print("Crawling error, No : ", num)
            is_success = False
            return whole_dict, is_success, standard
    
        driver.implicitly_wait(100)
    
    std_idx = 1    
    is_success = True
    return whole_dict, is_success, std_idx


# 페이지 넘기는 크롤러
def page_crawler(start_page, chrome_path, url, std_idx):
    driver = webdriver.Chrome(executable_path=chrome_path)
    driver.get(url)
    
    driver.find_element(By.CLASS_NAME, "last").click()
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    last_page = soup.find_all("a", {"class": "on"})[1].text
    driver.find_element(By.CLASS_NAME, "first").click()
    
    std_dict = {}
    std_dict['stds'] = []
    
    start_page = int(start_page)
    if start_page % 10 == 0:
        num_next = start_page // 10 - 1
    else:
        num_next = start_page // 10

    for i in range(num_next):
        driver.find_element(By.CLASS_NAME, 'next').click()
        
    page_source = driver.page_source
    lst_page_soup = BeautifulSoup(page_source, 'html.parser')

    # 페이지 버튼 갯수
    page_button_data = lst_page_soup.find("div", {'class':"page"})
    li_lst = page_button_data.find_all("li")
    li_lst = [button.text for button in li_lst]
    start = li_lst.index(str(start_page))
    
    is_success = False
    page_num = 0
    
    try:
        while True:
            for page in range(start + 1, len(li_lst) - 1):
                driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[3]/div/div/ul/li[{}]/a'.format(page)).click()
                page_num = driver.find_element(By.XPATH, '//*[@id="tabs-container"]/div[3]/div/div/ul/li[{}]/a'.format(page)).text
                
                page_source = driver.page_source
                lst_page_soup = BeautifulSoup(page_source, 'html.parser')
                driver.implicitly_wait(100)
                
                std_dict, is_success, std_idx = std_crawler(std_dict, lst_page_soup, driver, std_idx)
                
                if page_num == last_page:
                    is_success = True
                    return std_dict, int(page_num), is_success, std_idx
                
                elif is_success == False:
                    std_dict_false, page_num_false, is_success_false, std_idx_false = page_crawler(int(page_num), chrome_path, url, std_idx)
                    for std in std_dict_false['stds']:
                      std_dict['stds'].append(std)

            # 다시 1페이지부터
            start = 2
            
            driver.find_element(By.CLASS_NAME, 'next').click()
            
    except:
        return std_dict, int(page_num), is_success, std_idx
    

# json 파일 dataframe으로 읽는 함수
def json_reader(filename):
    with open (filename, "r", encoding = 'UTF8') as f:
        data = json.load(f)
    
    df = pd.json_normalize(data['stds'])
    return df


# DB에 테이블이 생성되어 있다면 테이블에 데이터 넣는 함수
def db_process(filename, host, password, schema_name, table_name):
    try:
        db = pymysql.connect(host = host, port = 3306, user = 'root', password = password,
                        db = schema_name, charset = 'utf8')   # charset: 인코딩 설정
    except: 
        print("Can not connent to DB server. Please check the connection")
        return
    
    cursor = db.cursor()

    sql = 'INSERT INTO ' + table_name + ' (doc_num, doc_name_ko, doc_name_en, publish_date, final_date, json_data, crawled_time, tag) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'

    output_df = json_reader(filename)
    output_df = output_df.reset_index()

    with open (filename, "r", encoding = 'UTF8') as f:
        data = json.load(f)
        
    lst = data['stds']
            
    for i in range(len(output_df)):
        json_data = json.dumps(lst[i])
        try:
            cursor.execute(sql, (output_df.loc[i]["표준번호"], output_df.loc[i]["표준명(한글)"], output_df.loc[i]["표준명(영문)"], output_df.loc[i]["제정일"], output_df.loc[i]["최종개정확인일"], json_data, output_df.loc[i]["crawled_time"], filename[7:15]))
        except:
            if len(output_df.loc[i]["제정일"]) == 0:
                cursor.execute(sql, (output_df.loc[i]["표준번호"], output_df.loc[i]["표준명(한글)"], output_df.loc[i]["표준명(영문)"], None, output_df.loc[i]["최종개정확인일"], json_data, output_df.loc[i]["crawled_time"], filename[7:15]))
            else:
                print("There is a problem while inserting data number : ", i)
    db.commit()

    db.close()
    print("Done with inserting data to DB")
    return


# main 함수 = crawling --> json data --> DB 연동 및 데이터 inserting
def main(last_page_no, max_retry_num, chrome_path, url, host, password, schema_name, table_name):
    data_list = {}
    data_list['stds'] = []
    current_retry_count = 0
    std_idx = 1
    now = datetime.datetime.now()
    output_name = "output_{}.json".format(now.strftime('%Y%m%d'))
    
    while True:
        if current_retry_count >= max_retry_num:
            print("number of retry exceeds maximum retry number")
            print("last page of crawling is ", last_page_no)
            transform_to_json(data_list, output_name)
            break
            # return data_list, is_success, current_retry_count
        
        current_data_list, end_page_no, is_success, std_idx = page_crawler(last_page_no, chrome_path, url, std_idx)
        
        # merge current_data_list to data_list 
        for i in current_data_list['stds']:
            if i not in data_list['stds']:
                data_list['stds'].append(i)
                
        # if it couldn't reach the last page
        if is_success == False:
            current_retry_count = current_retry_count + 1
            transform_to_json(data_list, output_name)
            last_page_no = end_page_no
            print("Reconnecting to the page number : ", last_page_no)
        
        # if it reaches the last page
        elif is_success:
            print("Done with crawling")
            transform_to_json(data_list, output_name)
            break
            # return data_list, is_success, current_retry_count
            
    return db_process(output_name, host, password, schema_name, table_name)
        
        
url = "https://standard.go.kr/KSCI/standardIntro/getStandardSearchList.do?menuId=919&topMenuId=502"
main(890, 10, r"C:\Users\gihok\chatbot\chromedriver.exe", url, "192.168.0.124", "linuxer", "std_crawled_data", "std_data_check")


<ipython-input-10-45dd8dbc645d>:163: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path)


Crawling error, No :  8894
Reconnecting to the page number :  890


WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x009078B3+2193587]
	Ordinal0 [0x008A0681+1771137]
	Ordinal0 [0x007B41A8+803240]
	Ordinal0 [0x007A7E1E+753182]
	Ordinal0 [0x007A7A0F+752143]
	Ordinal0 [0x007A6FAF+749487]
	Ordinal0 [0x007A5E94+745108]
	Ordinal0 [0x007A6498+746648]
	Ordinal0 [0x007B0610+787984]
	Ordinal0 [0x007A781D+751645]
	Ordinal0 [0x007A8450+754768]
	Ordinal0 [0x007A7A2A+752170]
	Ordinal0 [0x007A6FAF+749487]
	Ordinal0 [0x007A5E94+745108]
	Ordinal0 [0x007A636D+746349]
	Ordinal0 [0x007B5974+809332]
	Ordinal0 [0x0080E18D+1171853]
	Ordinal0 [0x007FD7AC+1103788]
	Ordinal0 [0x0080DAE2+1170146]
	Ordinal0 [0x007FD5C6+1103302]
	Ordinal0 [0x007D77E0+948192]
	Ordinal0 [0x007D86E6+952038]
	GetHandleVerifier [0x00BB0CB2+2738370]
	GetHandleVerifier [0x00BA21B8+2678216]
	GetHandleVerifier [0x009917AA+512954]
	GetHandleVerifier [0x0